<a href="https://colab.research.google.com/github/armandoordonez/GenAI/blob/main/Lab_2_fine_tune_generative_ai_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tune un modelo de Gen AI Model para resúmenes

En este cuaderno, se hará en fine-tune de un LLM existente de Hugging Face para mejorar el resumen de los diálogos. Utilizará el modelo [FLAN-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5), que proporciona un modelo tuneado con instrucciones de alta calidad y puede resumir texto. Para mejorar las inferencias, se usará full fine-tuning y evaluará los resultados con métricas de ROUGE. Luego, se usará Parameter Efficient Fine-Tuning (PEFT), evaluará el modelo resultante y verá que los beneficios de PEFT superan las desventajas de rendimiento.

# Tabla de contenido

- [ 1 - Set up Kernel, Load Required Dependencies, Dataset and LLM](#1)
  - [ 1.1 - Set up Kernel and Required Dependencies](#1.1)
  - [ 1.2 - Load Dataset and LLM](#1.2)
  - [ 1.3 - Test the Model with Zero Shot Inferencing](#1.3)
- [ 2 - Perform Full Fine-Tuning](#2)
  - [ 2.1 - Preprocess the Dialog-Summary Dataset](#2.1)
  - [ 2.2 - Fine-Tune the Model with the Preprocessed Dataset](#2.2)
  - [ 2.3 - Evaluate the Model Qualitatively (Human Evaluation)](#2.3)
  - [ 2.4 - Evaluate the Model Quantitatively (with ROUGE Metric)](#2.4)
- [ 3 - Perform Parameter Efficient Fine-Tuning (PEFT)](#3)
  - [ 3.1 - Setup the PEFT/LoRA model for Fine-Tuning](#3.1)
  - [ 3.2 - Train PEFT Adapter](#3.2)
  - [ 3.3 - Evaluate the Model Qualitatively (Human Evaluation)](#3.3)
  - [ 3.4 - Evaluate the Model Quantitatively (with ROUGE Metric)](#3.4)

<a name='1'></a>
## 1 - Set up Kernel, Load Required Dependencies, Dataset and LLM

In [ ]:
!python --version

Python 3.12.11


In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"  # 👈 debe ir primero
# ------------------------------
# Instalación para GPU T4 en Colab
# ------------------------------

print("🚀 Instalando PyTorch con soporte CUDA...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

print("🚀 Instalando Transformers...")
!pip install transformers datasets evaluate rouge_score peft accelerate huggingface_hub safetensors

# ------------------------------
# Verificación
# ------------------------------
print("\n🔍 Verificando instalaciones...")
try:
    import torch
    import transformers
    import datasets
    import evaluate
    import peft

    print("✅ Verificación exitosa:")
    print(f"   PyTorch: {torch.__version__}")
    print(f"   CUDA disponible: {torch.cuda.is_available()}")
    print(f"   Transformers: {transformers.__version__}")
    print(f"   Datasets: {datasets.__version__}")
    print(f"   PEFT: {peft.__version__}")

except ImportError as e:
    print(f"❌ Error en la verificación: {e}")
    print("Recuerda reiniciar el kernel después de instalar las librerías.")


🚀 Instalando PyTorch con soporte CUDA...
Looking in indexes: https://download.pytorch.org/whl/cu121
🚀 Instalando Transformers...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.9 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6f0b5b3aa5888efdec19e3e131c744db81021bc7f01efa581e299520c86df03b
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score

🔍 Verificando instalaciones...
✅ Verificación exitosa:
   PyTorch: 2.8.0+cu126
   CUDA disponible: False
   Transformers: 4.56.1
   Datasets: 4.0.0
   PEFT: 0.17.1


In [2]:
!pip show torch torchdata transformers datasets evaluate rouge_score loralib peft

Name: torch
Version: 2.8.0+cu126
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.12/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-cufile-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, setuptools, sympy, triton, typing-extensions
Required-by: accelerate, fastai, peft, sentence-transformers, timm, torchaudio, torchdata, torchvision
---
Name: torchdata
Version: 0.11.0
Summary: Composable data loading modules for PyTorch
Home-page: https://github.com/pytorch/data
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD
Location: /usr/local/lib/pyth

In [3]:
# verificar que transformers funciona

from transformers import T5ForConditionalGeneration, T5Tokenizer

# Probar con el modelo más pequeño

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Test básico
input_text = "translate English to Spanish: Hello world"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids, max_length=20)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"✅ T5 funciona: {result}")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

✅ T5 funciona: Hallo Welt


<a name='1.1'></a>
### 1.1 - Set up Kernel and Required Dependencies

In [4]:
# Importe los componentes necesarios.
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

<a name='1.2'></a>
### 1.2 - Cargar Dataset y LLM

[Medical_NLI](https://huggingface.co/datasets/OmKumbhare2002/medical_NLI_dataset_test) es un dataset de Hugging Face con más de 14k registros con hipótesis y premisas donde se busca determinar la relación entre ambos enunciados implicación (entailment), contradicción (contradiction) y neutralidad (neutral). El dataset tiene una version convesational que hace uso de prompt engineering y trae un formato de respuesta y por otro lado el processed que es solo el texto sin alguna instrucción adicional los cuales vamos a usar para hacer comparativas, prompt engineering original vs fine tunning sin prompt engineering

In [5]:
# Dataset sin promptear
dataset = load_dataset("araag2/MedNLI", "processed")
# Dataset prompteado
with_prompt = load_dataset("araag2/MedNLI", "conversational")

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/637k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/84.2k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/83.1k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11232 [00:00<?, ? examples/s]

...

dev-00000-of-00001.parquet:   0%|          | 0.00/154k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11232 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1395 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1422 [00:00<?, ? examples/s]

In [6]:
print(dataset["train"].features)

{'id': Value('string'), 'Label': Value('string'), 'Premise': Value('string'), 'Hypothesis': Value('string')}


Load the pre-trained [FLAN-T5-SMALL model](https://huggingface.co/google/flan-t5-small) and its tokenizer directly from HuggingFace. Notice that you will be using the [small version](https://huggingface.co/google/flan-t5-small) of FLAN-T5. Setting `torch_dtype=torch.bfloat16` specifies the memory type to be used by this model.

In [7]:
# Cargamos el modelo y el tokenizador

model_name='google/flan-t5-small'
# model_name='t5-base'
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Es posible extraer la cantidad de parámetros del modelo y descubrir cuántos de ellos se pueden entrenar.

In [8]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"Parametros entrenables del modelo: {trainable_model_params:,.0f}\n Total de parametros del modelo: {all_model_params:,.0f}\n Porcentaje de parametros entrenables {100 * trainable_model_params / all_model_params:.0f}%"
#print(f'El area es: {area:,.2f}')
print(print_number_of_trainable_model_parameters(original_model))

Parametros entrenables del modelo: 76,961,152
 Total de parametros del modelo: 76,961,152
 Porcentaje de parametros entrenables 100%


En comparación al FLAN-T5-BASE pasamos de un alrededor de 250 millones de parámetros a alrededor de 80. Esto nos permite poder realizar un fine tunning de todo el modelo en tan solo 15 minutos con la tier gratis de colab que ofrece la CPU y 12.7 gigas de RAM

<a name='1.3'></a>
### 1.3 - Prueba del modelo con Zero Shot Inferencing

Pruebe el modelo con la inferencia de tiro cero. Puede ver que el modelo tiene dificultades para resumir el diálogo en comparación con el resumen de referencia, pero extrae información importante del texto que indica que el modelo se puede ajustar a la tarea en cuestión.

In [9]:
# Con el dataset que ya no trae prompt engineering
index = 100

hypothesis = dataset['train'][index]['Hypothesis']
premise = dataset['train'][index]['Premise']
label = dataset['train'][index]['Label']

prompt = f"""
Determine the relationship between the next statements:

Premise: {premise}

Hypothesis: {hypothesis}

Classify between: entailment, neutral and contradiction:
"""

inputs = tokenizer(str(prompt), return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'PROMPT DE ENTRADA:\n{prompt}')
print(dash_line)
print(f'RESPUESTA HUMANA (BASELINE) :\n{label}\n')
print(dash_line)
print(f'RESPUESTA GENERADA POR EL MODELO CON ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
PROMPT DE ENTRADA:

Determine the relationship between the next statements:

Premise: Came to ED complaining of vomiting and weakness.

Hypothesis:  Patient has negative ROS

Classify between: entailment, neutral and contradiction:

---------------------------------------------------------------------------------------------------
RESPUESTA HUMANA (BASELINE) :
contradiction

---------------------------------------------------------------------------------------------------
RESPUESTA GENERADA POR EL MODELO CON ZERO SHOT:
entailment


In [10]:
# Con prompt engineering / soft prompting
index = 300

prompt = with_prompt['train'][index]['prompt']
completion = with_prompt['train'][index]['completion']

inputs = tokenizer(str(prompt), return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'PROMPT DE ENTRADA (PROMPT ENGINEERING):\n{prompt}')
print(dash_line)
print(f'RESPUESTA HUMANA (BASELINE) :\n{completion}\n')
print(dash_line)
print(f'RESPUESTA GENERADA POR EL MODELO CON ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
PROMPT DE ENTRADA (PROMPT ENGINEERING):
[{'role': 'user', 'content': 'You are a medical expert tasked with determining the relationship between a medical premise and a hypothesis.The **Medical Premise:** is a medical statement, and the **Hypothesis:** is a medical hypothesis about what can be infered from that statement. The task is to classify the relationship between the premise and the hypothesis as one of the following: entailment, neutral, or contradiction. \n\n**Medical Premise:** His O2Sat remains above 90% on room air but appears to drop when patient falls asleep.\n\n**Hypothesis:**  Patient has OND\n\nPlease provide your judgement (entailment, neutral or contradiction), corresponding to the correct option that associates the premise and hypothesis. Be as accurate as possible.\n'}]
---------------------------------------------------------------------------------------------------

El modelo con prompt engineering es significativamente superior al otro. Aunque cometió un error al clasificiar realmente tardé varios minutos para que clasifique erroneamente, esto se debe a la forma en que el prompt le otorga:
 * rol: experto médico
 * objetivo: determinar la relación entre la premisa e hipótesis
 * entrada: JSON con formato de chat
 * formato de salida: seleción entre [entailment, neutral, contradiction]
 * criterios de calidad: se lo más preciso posible

 Mientras que al pasarle la información sin prompt engineering observamos muy rápidamente como la calidad de las respuestas decae, incluso si no incluimos las opciones de respuesta en el prompt otorga resultados como que es imposible determinarlo

<a name='2'></a>
## 2 - Realizar Full Fine-Tuning

<a name='2.1'></a>
### 2.1 - Preprocesar el dataset Medical NLI

Se necesita convertir las ternas premisa-hipótesis-conclusón en instrucciones explícitas para el LLM. Agregar una instrucción al inicio del dialogo como `Determine the relationship between the next statements` y al inicio de la conclución agregar `Classify between: entailment, neutral and contradiction`como se muestra a continuación:

Training prompt:
```
Determine the relationship between the next statements:

Premise: His O2Sat remains above 90% on room air but appears to drop when patient falls asleep.

Hypothesis:  Patient has OND

Classify between: entailment, neutral and contradiction:
```

Training response:
```
entailment
```

Then preprocess the prompt-response dataset into tokens and pull out their `input_ids` (1 per token).

In [11]:
dataset.column_names

{'train': ['id', 'Label', 'Premise', 'Hypothesis'],
 'dev': ['id', 'Label', 'Premise', 'Hypothesis'],
 'test': ['id', 'Label', 'Premise', 'Hypothesis']}

In [12]:
def tokenize_function(example):
    start_prompt = 'Determine the relationship between the next statements:\n\nPremise: '
    middle_prompt = '\n\nHypothesis: '
    end_prompt = '\n\nClassify between: entailment, neutral and contradiction: '
    prompt = [start_prompt + hypothesis + middle_prompt + premise + end_prompt for hypothesis, premise in zip(example["Hypothesis"], example["Premise"])]
    print("Size of prompt list: ", len(prompt))  # Imprimir el tamaño de la lista
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer([str(completion) for completion in example["Label"]], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(['id', 'Hypothesis', 'Premise', 'Label'])

Map:   0%|          | 0/11232 [00:00<?, ? examples/s]

Size of prompt list:  1000
Size of prompt list:  1000
Size of prompt list:  1000
Size of prompt list:  1000
Size of prompt list:  1000
Size of prompt list:  1000
Size of prompt list:  1000
Size of prompt list:  1000
Size of prompt list:  1000
Size of prompt list:  1000
Size of prompt list:  1000
Size of prompt list:  232


Map:   0%|          | 0/1395 [00:00<?, ? examples/s]

Size of prompt list:  1000
Size of prompt list:  395


Map:   0%|          | 0/1422 [00:00<?, ? examples/s]

Size of prompt list:  1000
Size of prompt list:  422


In [13]:
print (type(tokenized_datasets))
first_example = tokenized_datasets["train"].select([0])
print(first_example)
print(first_example['labels'])

<class 'datasets.dataset_dict.DatasetDict'>
Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 1
})
Column([[3, 35, 5756, 297, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

Compruebe las formas de las tres partes del conjunto de datos:

In [14]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['dev'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (11232, 2)
Validation: (1395, 2)
Test: (1422, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 11232
    })
    dev: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1395
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 1422
    })
})


El dataset de salida esta listo para el fine-tunning.

<a name='2.2'></a>
### 2.2 - Aplicar Fine -Tunning para el modelo con el dataset Preprocesado

Ahora utilice la clase integrada "Trainer" de Hugging Face (consulte la documentación [aqui](https://huggingface.co/docs/transformers/main_classes/trainer)). Pase el conjunto de datos preprocesado con referencia al modelo original. Los demás parámetros de entrenamiento se encuentran experimentalmente y no es necesario profundizar en ellos por el momento.

In [15]:
output_dir = f'./med-nli-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['dev']
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Start training process...



In [16]:
print(tokenized_datasets.column_names)

{'train': ['input_ids', 'labels'], 'dev': ['input_ids', 'labels'], 'test': ['input_ids', 'labels']}


In [17]:
print(output_dir)

./med-nli-training-1758567606


In [ ]:
# # Tarda 15 minutos pero el checkpoint ya existe dentro de la carpeta del repositorio
# import os
# os.environ["WANDB_DISABLED"] = "true"
# trainer.train()

Step,Training Loss
1,63.750000


TrainOutput(global_step=1, training_loss=63.75, metrics={'train_runtime': 947.1653, 'train_samples_per_second': 0.008, 'train_steps_per_second': 0.001, 'total_flos': 1487124037632.0, 'train_loss': 63.75, 'epoch': 0.0007122507122507123})

Create an instance of the `AutoModelForSeq2SeqLM` class for the instruct model:

Entrenar el modelo tarda 15 minutos. Sin embargo, el checkpoint lo puedes encontrar en el repositorio para ahorrarte ese tiempo y poder probar si así lo deseas. Recuerda es un modelo pequeño y el conjunto de entrenamiento también lo es. Una versión completamente tuneada del modelo original toma horas en una GPU.

Lo más destacable es el training loss ya que nos indica cómo se comporta nuestro dataset frente a la partición de validación (dev) como podemos observar todavía hay un gran margen de mejora ya que tiene una perdida de casi el 64% esto se puede deber a que estamos tratando de hacer fine tunning a un LLM con tan solo 11k datos y que únicamente le permitimos dar 1 paso (una actualización de parámetros) por lo que en una sola ida y vuelta no se logra recuperar la suficiente información. Sin embargo esta es una limitación impuesta por los límites computacionales en el hambiente gratuito de colab

In [18]:
# Está en el repo
instruct_model_name='./med-nli-training/checkpoint-1'
instruct_model = AutoModelForSeq2SeqLM.from_pretrained(instruct_model_name, torch_dtype=torch.bfloat16)

`torch_dtype` is deprecated! Use `dtype` instead!


In [19]:
type(instruct_model)

transformers.models.t5.modeling_t5.T5ForConditionalGeneration

<a name='2.3'></a>
### 2.3 - Evaluar el modelo cualitativamente (evaluación humana)

Como ocurre con muchas aplicaciones de IA, un enfoque cualitativo en el que uno se hace la pregunta "¿Mi modelo se comporta como se supone que debe hacerlo" suele ser un buen punto de partida. En el siguiente ejemplo, podemos ver como al probar de nuevo con el mismo prompt en el que el modelo falla previamente ahora lo responde de forma correcta para algunos de los casos. Sin embargo al compararlo con el de prompt engineering en la mayoría de casos se optiene la misma respuesta.

In [137]:
# Sin prompt engineering
index = 3100
hypothesis = dataset['train'][index]['Hypothesis']
premise = dataset['train'][index]['Premise']
label = dataset['train'][index]['Label']

prompt = f"""
Determine the relationship between the next statements:

Premise: {premise}

Hypothesis: {hypothesis}

Classify between: entailment, neutral and contradiction
"""

inputs = tokenizer(prompt, return_tensors='pt')

original_model_outputs = original_model.generate(inputs["input_ids"], max_new_tokens=200)
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(inputs["input_ids"], max_new_tokens=200)
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'RESPUESTA HUMANA (BASELINE):\n{label}')
print(dash_line)
print(f'MODELO ORIGINAL:\n{original_model_text_output}')
print(dash_line)
print(f'MODELO FINE TUNEADO:\n{instruct_model_text_output}')

---------------------------------------------------------------------------------------------------
RESPUESTA HUMANA (BASELINE):
contradiction
---------------------------------------------------------------------------------------------------
MODELO ORIGINAL:
entailment
---------------------------------------------------------------------------------------------------
MODELO FINE TUNEADO:
contradiction


In [270]:
# Con prompt engineering
index = 11000

prompt = with_prompt['train'][index]['prompt']
completion = with_prompt['train'][index]['completion']

inputs = tokenizer(str(prompt), return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

original_model_outputs = original_model.generate(inputs["input_ids"], max_new_tokens=200)
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(inputs["input_ids"], max_new_tokens=200)
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'RESPUESTA HUMANA (BASELINE):\n{completion}')
print(dash_line)
print(f'MODELO ORIGINAL:\n{original_model_text_output}')
print(dash_line)
print(f'MODELO FINE TUNEADO:\n{instruct_model_text_output}')

---------------------------------------------------------------------------------------------------
RESPUESTA HUMANA (BASELINE):
[{'role': 'assistant', 'content': '**Answer:** neutral\n'}]
---------------------------------------------------------------------------------------------------
MODELO ORIGINAL:
neutral
---------------------------------------------------------------------------------------------------
MODELO FINE TUNEADO:
neutral


<a name='2.4'></a>
### 2.4 - Evaluar el modelo cuantitativamente (con la métrica ROUGE)

[ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) Ayuda a cuantificar la validez de los resúmenes generados por los modelos. Compara los resúmenes con un resumen de referencia, generalmente creado por un usuario. Si bien no es perfecto, indica el aumento general en la eficacia del resumen que hemos logrado mediante el ajuste.

In [228]:
rouge = evaluate.load('rouge')

Genere las salidas para la muestra del conjunto de datos de prueba (solo 10 prompts) y guarde los resultados.

In [229]:
# Tomamos 10 ejemplos de test para comparar
premises = dataset['test'][0:10]['Premise']
hypotheses = dataset['test'][0:10]['Hypothesis']
human_baseline_labels = dataset['test'][0:10]['Label']

original_model_preds = []
instruct_model_preds = []

for premise, hypothesis in zip(premises, hypotheses):
    prompt = f"""
Determine the relationship between the next statements:

Premise: {premise}

Hypothesis: {hypothesis}

Classify between: entailment, neutral and contradiction
"""
    inputs = tokenizer(prompt, return_tensors="pt")

    # Modelo original
    original_model_outputs = original_model.generate(inputs["input_ids"], max_new_tokens=200)
    original_text = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_preds.append(original_text)

    # Modelo fine-tuneado
    instruct_model_outputs = instruct_model.generate(inputs["input_ids"], max_new_tokens=200)
    instruct_text = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_preds.append(instruct_text)

# Unimos baseline + predicciones
zipped_preds = list(zip(human_baseline_labels, original_model_preds, instruct_model_preds))

df = pd.DataFrame(
    zipped_preds,
    columns=['human_baseline_labels', 'original_model_preds', 'instruct_model_preds']
)

df

,human_baseline_labels,original_model_preds,instruct_model_preds
0,entailment,entailment,entailment
1,contradiction,entailment,entailment
2,neutral,entailment,entailment
3,entailment,entailment,entailment
4,contradiction,contradiction,contradiction
5,neutral,entailment,entailment
6,entailment,entailment,entailment
7,contradiction,entailment,entailment
8,neutral,entailment,entailment
9,entailment,entailment,entailment


Evalúe los modelos que calculan las métricas ROUGE. ¡Observe la mejora en los resultados!

In [230]:
original_model_results = rouge.compute(
    predictions=original_model_preds,
    references=human_baseline_labels[0:len(original_model_preds)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_preds,
    references=human_baseline_labels[0:len(instruct_model_preds)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)

print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.5), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.5), 'rougeLsum': np.float64(0.5)}
INSTRUCT MODEL:
{'rouge1': np.float64(0.5), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.5), 'rougeLsum': np.float64(0.5)}


Para evaluar una sección más amplia del modelo usaremos los primeros 700 registros por cuestiones computacionales y de tiempo (calcular los 11k tardaría más de 2 horas). Podrás encontrar el csv con los 700 primeros ejemplos en github

In [ ]:
# # Tomamos 700 ejemplos de test para comparar
# premises = dataset['test'][0:700]['Premise']
# hypotheses = dataset['test'][0:700]['Hypothesis']
# human_baseline_labels = dataset['test'][0:700]['Label']

# original_model_preds = []
# instruct_model_preds = []

# for premise, hypothesis in zip(premises, hypotheses):
#     prompt = f"""
# Determine the relationship between the next statements:

# Premise: {premise}

# Hypothesis: {hypothesis}

# Classify between: entailment, neutral and contradiction
# """
#     inputs = tokenizer(prompt, return_tensors="pt")

#     # Modelo original
#     original_model_outputs = original_model.generate(inputs["input_ids"], max_new_tokens=200)
#     original_text = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
#     original_model_preds.append(original_text)

#     # Modelo fine-tuneado
#     instruct_model_outputs = instruct_model.generate(inputs["input_ids"], max_new_tokens=200)
#     instruct_text = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
#     instruct_model_preds.append(instruct_text)

# # Unimos baseline + predicciones
# zipped_preds = list(zip(human_baseline_labels, original_model_preds, instruct_model_preds))

# results = pd.DataFrame(
#     zipped_preds,
#     columns=['human_baseline_labels', 'original_model_preds', 'instruct_model_preds']
# )

# results

,human_baseline_labels,original_model_preds,instruct_model_preds
0,entailment,neutral,entailment
1,contradiction,neutral,entailment
2,neutral,entail,entailment
3,entailment,neutral,entailment
4,contradiction,entailment,contradiction
...,...,...,...
695,neutral,entailment,entailment
696,entailment,entailment,entailment
697,contradiction,entailment,entailment
698,neutral,entailment,entailment


In [310]:
results = pd.read_csv("data/nli-results-all.csv")

human_baseline_summaries = results['human_baseline_labels'].values
original_model_summaries = results['original_model_preds'].values
instruct_model_summaries = results['instruct_model_preds'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.32057142857142856), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.32142857142857145), 'rougeLsum': np.float64(0.3211428571428572)}
INSTRUCT MODEL:
{'rouge1': np.float64(0.37714285714285717), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.38), 'rougeLsum': np.float64(0.37857142857142856)}


The results show a small improvement in some ROUGE metrics:

In [311]:
print("Absolute percentage improvement of INSTRUCT MODEL over HUMAN BASELINE")

improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of INSTRUCT MODEL over HUMAN BASELINE
rouge1: 5.66%
rouge2: 0.00%
rougeL: 5.86%
rougeLsum: 5.74%


<a name='3'></a>
## 3 - Fine tunning Eficiente de Parámetros (PEFT)

Ahora, realicemos un ajuste fino **PEFT** en lugar del ajuste fino completo, como se hizo anteriormente. PEFT es una forma de ajuste fino de instrucciones mucho más eficiente que el ajuste fino completo, con resultados de evaluación comparables, como verá pronto.

PEFT es un término genérico que incluye **Adaptación de Bajo Rango (LoRA)** y ajuste de prompts (¡que NO ES LO MISMO que ingeniería de prompts!). En la mayoría de los casos, cuando alguien habla de PEFT, se refiere a LoRA. LoRA, a un nivel muy alto, permite al usuario ajustar su modelo utilizando menos recursos computacionales (en algunos casos, una sola GPU). Después del ajuste fino para una tarea, caso de uso o inquilino específico con LoRA, el resultado es que el LLM original permanece sin cambios y surge un nuevo "adaptador LoRA". Este adaptador LoRA es mucho más pequeño que el LLM original: aproximadamente un porcentaje de un solo dígito del tamaño del LLM original (MB vs. GB).

No obstante, en el momento de la inferencia, el adaptador LoRA debe reunirse y combinarse con su LLM original para atender la solicitud de inferencia. La ventaja, sin embargo, es que muchos adaptadores LoRA pueden reutilizar el LLM original, lo que reduce los requisitos generales de memoria al atender múltiples tareas y casos de uso.

<a name='3.1'></a>


### 3.1 - Configuración del modelo PEFT/LoRA para el ajuste fino

Debe configurar el modelo PEFT/LoRA para el ajuste fino con un nuevo adaptador de capa/parámetro. Al usar PEFT/LoRA, se congela el LLM subyacente y solo se entrena el adaptador. Observe la configuración de LoRA a continuación. Observe el hiperparámetro de rango (`r`), que define el rango/dimensión del adaptador que se va a entrenar.

In [260]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)

Add LoRA adapter layers/parameters to the original LLM to be trained.

In [261]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

Parametros entrenables del modelo: 1,376,256
 Total de parametros del modelo: 78,337,408
 Porcentaje de parametros entrenables 2%


<a name='3.2'></a>
### 3.2 - Train PEFT Adapter

Define training arguments and create `Trainer` instance.

In [262]:
output_dir = f'./peft-med-nli-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=1,
    logging_steps=1,
    max_steps=1
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
)

print(output_dir)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


./peft-med-nli-training-1758573482


Now everything is ready to train the PEFT adapter and save the model.



In [263]:
# # Tarda 15 minutos
# peft_trainer.train()
# peft_model_path="./peft-med-nli-checkpoint-local"
# peft_trainer.model.save_pretrained(peft_model_path)
# tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
1,66.500000


('./peft-med-nli-checkpoint-local/tokenizer_config.json',
 './peft-med-nli-checkpoint-local/special_tokens_map.json',
 './peft-med-nli-checkpoint-local/spiece.model',
 './peft-med-nli-checkpoint-local/added_tokens.json',
 './peft-med-nli-checkpoint-local/tokenizer.json')

That training was performed on a subset of data. To load a fully trained PEFT model, read a checkpoint of a PEFT model from S3.

Comprueba que el tamaño de este modelo es mucho menor que el LLM original:

In [264]:
!dir ./peft-med-nli-checkpoint-local

adapter_config.json	   special_tokens_map.json  tokenizer.json
adapter_model.safetensors  spiece.model
README.md		   tokenizer_config.json


Prepare este modelo añadiendo un adaptador al modelo FLAN-T5 original. Está configurando `is_trainable=False` porque el plan es realizar inferencia únicamente con este modelo PEFT. Si estuviera preparando el modelo para entrenamiento posterior, debería configurar `is_trainable=True`.

In [268]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       './peft-med-nli-checkpoint-local/',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

The number of trainable parameters will be `0` due to `is_trainable=False` setting:

In [269]:
print(print_number_of_trainable_model_parameters(peft_model))

Parametros entrenables del modelo: 0
 Total de parametros del modelo: 78,337,408
 Porcentaje de parametros entrenables 0%


<a name='3.3'></a>
### 3.3 - Evaluate the Model Qualitatively (Human Evaluation)

Make inferences for the same example as in sections [1.3](#1.3) and [2.3](#2.3), with the original model, fully fine-tuned and PEFT model.

In [280]:
# Sin prompt engineering
index = 3100
hypothesis = dataset['train'][index]['Hypothesis']
premise = dataset['train'][index]['Premise']
label = dataset['train'][index]['Label']

prompt = f"""
Determine the relationship between the next statements:

Premise: {premise}

Hypothesis: {hypothesis}

Classify between: entailment, neutral and contradiction
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{label}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
contradiction
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
entailment
---------------------------------------------------------------------------------------------------
INSTRUCT MODEL:
contradiction
---------------------------------------------------------------------------------------------------
PEFT MODEL: contradiction


<a name='3.4'></a>
### 3.4 - Evaluate the Model Quantitatively (with ROUGE Metric)
Perform inferences for the sample of the test dataset (only 10 dialogues and summaries to save time).

In [304]:
premises = dataset['test'][0:10]['Premise']
hypotheses = dataset['test'][0:10]['Hypothesis']
human_baseline_labels = dataset['test'][0:10]['Label']

original_model_preds = []
instruct_model_preds = []
peft_model_preds = []

for premise, hypothesis in zip(premises, hypotheses):
    prompt = f"""
    Determine the relationship between the next statements:

    Premise: {premise}

    Hypothesis: {hypothesis}

    Classify between: entailment, neutral and contradiction
    """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    # Original
    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    # Tunning
    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    # Peft
    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_preds.append(original_model_text_output)
    instruct_model_preds.append(instruct_model_text_output)
    peft_model_preds.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_labels, original_model_preds, instruct_model_preds, peft_model_preds))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_labels', 'original_model_preds', 'instruct_model_preds', 'peft_model_preds'])
df

,human_baseline_labels,original_model_preds,instruct_model_preds,peft_model_preds
0,entailment,contradiction,entailment,entailment
1,contradiction,entailment,entailment,entailment
2,neutral,contradiction,entailment,entailment
3,entailment,neutral,entailment,entailment
4,contradiction,contradiction,contradiction,contradiction
5,neutral,entail,entailment,entailment
6,entailment,entailment,entailment,entailment
7,contradiction,entailment,entailment,entailment
8,neutral,entailment,entailment,entailment
9,entailment,entailment,entailment,entailment



Calcule la puntuación ROUGE para este subconjunto de los datos.

In [312]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_preds,
    references=human_baseline_labels[0:len(original_model_preds)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_preds,
    references=human_baseline_labels[0:len(instruct_model_preds)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_preds,
    references=human_baseline_labels[0:len(peft_model_preds)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.32057142857142856), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.32142857142857145), 'rougeLsum': np.float64(0.3211428571428572)}
INSTRUCT MODEL:
{'rouge1': np.float64(0.37714285714285717), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.38), 'rougeLsum': np.float64(0.37857142857142856)}
PEFT MODEL:
{'rouge1': np.float64(0.38), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.38142857142857145), 'rougeLsum': np.float64(0.38142857142857145)}


Notice, that PEFT model results are not too bad, while the training process was much easier!

You already computed ROUGE score on the full dataset, after loading the results from the `data/dialogue-summary-training-results.csv` file. Load the values for the PEFT model now and check its performance compared to other models.

In [307]:
# Ya está en el .csv
# # Tomamos 700 ejemplos de test para comparar
# premises = dataset['test'][0:700]['Premise']
# hypotheses = dataset['test'][0:700]['Hypothesis']
# human_baseline_labels = dataset['test'][0:700]['Label']

# original_model_preds = []
# instruct_model_preds = []
# peft_model_preds = []

# for premise, hypothesis in zip(premises, hypotheses):
#     prompt = f"""
#     Determine the relationship between the next statements:

#     Premise: {premise}

#     Hypothesis: {hypothesis}

#     Classify between: entailment, neutral and contradiction
#     """

#     input_ids = tokenizer(prompt, return_tensors="pt").input_ids

#     # Original
#     original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
#     original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

#     # Tunning
#     instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
#     instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

#     # Peft
#     peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
#     peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

#     original_model_preds.append(original_model_text_output)
#     instruct_model_preds.append(instruct_model_text_output)
#     peft_model_preds.append(peft_model_text_output)

# # Unimos baseline + predicciones
# zipped_preds = list(zip(human_baseline_labels, original_model_preds, instruct_model_preds, peft_model_preds))

# results = pd.DataFrame(
#     zipped_preds,
#     columns=['human_baseline_labels', 'original_model_preds', 'instruct_model_preds', 'peft_model_preds']
# )

# results

,human_baseline_labels,original_model_preds,instruct_model_preds,peft_model_preds
0,entailment,neutral,entailment,entailment
1,contradiction,contradiction,entailment,entailment
2,neutral,entail,entailment,entailment
3,entailment,neutral,entailment,entailment
4,contradiction,entailment,contradiction,contradiction
...,...,...,...,...
695,neutral,contradiction,entailment,entailment
696,entailment,entailment,entailment,entailment
697,contradiction,entail,entailment,entailment
698,neutral,neutral,entailment,entailment


In [315]:
human_baseline_labels = results['human_baseline_labels'].values
original_model_preds = results['original_model_preds'].values
instruct_model_preds = results['instruct_model_preds'].values
peft_model_preds = results['peft_model_preds'].values

original_model_results = rouge.compute(
    predictions=original_model_preds,
    references=human_baseline_labels[0:len(original_model_preds)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_preds,
    references=human_baseline_labels[0:len(instruct_model_preds)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_results = rouge.compute(
    predictions=peft_model_preds,
    references=human_baseline_labels[0:len(peft_model_preds)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': np.float64(0.32057142857142856), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.32142857142857145), 'rougeLsum': np.float64(0.3211428571428572)}
INSTRUCT MODEL:
{'rouge1': np.float64(0.37714285714285717), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.38), 'rougeLsum': np.float64(0.37857142857142856)}
PEFT MODEL:
{'rouge1': np.float64(0.38), 'rouge2': np.float64(0.0), 'rougeL': np.float64(0.38142857142857145), 'rougeLsum': np.float64(0.38142857142857145)}


The results show even a better improvement over fine-tuning this is because our fine tunning was partial compared to PEFT. Usually PEFT offers worst results with the benefits of PEFT typically outweigh the slightly-lower performance metrics.

Calculate the improvement of PEFT over the original model:

In [316]:
print("Absolute percentage improvement of PEFT MODEL over HUMAN BASELINE")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over HUMAN BASELINE
rouge1: 5.94%
rouge2: 0.00%
rougeL: 6.00%
rougeLsum: 6.03%


Now calculate the improvement of PEFT over a full fine-tuned model:

In [317]:
print("Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL")

improvement = (np.array(list(peft_model_results.values())) - np.array(list(instruct_model_results.values())))
for key, value in zip(peft_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

Absolute percentage improvement of PEFT MODEL over INSTRUCT MODEL
rouge1: 0.29%
rouge2: 0.00%
rougeL: 0.14%
rougeLsum: 0.29%


Here you see a small percentage increase in the ROUGE metrics vs. full fine-tuned (Because of how thing went on). However, the training requires much less computing and memory resources (often just a single GPU).